## Data Transformation

In [4]:
import os
%pwd

'f:\\TextSummerizer\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'f:\\TextSummerizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [8]:
from src.TextSummerizer.constants import *
from src.TextSummerizer.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config['artifacts_root']])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [10]:
import os
from src.TextSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

### DAta Transformation component

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        dataset_samusm=load_from_disk(self.config.data_path)
        dataset_samusm_pt=dataset_samusm.map(self.convert_examples_to_features, batched=True)
        dataset_samusm_pt.save_to_disk(self.config.root_dir)


In [ ]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(data_transformation_config)
data_transformation.convert()

2025-09-03 16:50:04,529 - INFO - yaml file :config\config.yaml loaded successfully
2025-09-03 16:50:04,529 - INFO - yaml file :F:\TextSummerizer\params.yaml loaded successfully
2025-09-03 16:50:04,529 - INFO - created directory at :artifacts
2025-09-03 16:50:04,537 - INFO - created directory at :artifacts/data_transformation


f:\TextSummerizer\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hkiit\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]f:\TextSummerizer\venv\lib\site-packages\transformers\tokenizati